In [3]:
#Run the first time only
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

MemoryError: Unable to allocate 419. MiB for an array with shape (54950000,) and data type float64

In [2]:
X,y = mnist['data'],mnist['target']
X.shape

NameError: name 'mnist' is not defined

In [ ]:
y.shape

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
some_digit=X[0]
some_digit_img = some_digit.reshape(28,28)
plt.imshow(some_digit_img, cmap=mpl.cm.binary, interpolation="nearest")
plt.axis('off')
plt.show()

In [ ]:
y[0]

In [ ]:
y = y.astype(np.uint8)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


**Training a Binary Classifier**

In [ ]:
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)


In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf =SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)
sgd_clf.predict([some_digit])

**Performance Measures**

*Measuring Accuracy Using Cross-Validation*

-   Self Implemented Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state= 42)
for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold=y_train_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct/len(y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

-   Because only 10% of the data are 5s, if the machine only chooses no it will have more than 90% accuracy. 
-   Therefore, accuracy is generally not the prefered measure for classifiers, especially with skewed datasets

*Confusion Matrix*

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

-   Precision = TP/(TP+FP) = 83.7%
-   Recall (Sensitivity) = TP/(TP+FN) = 65.1%

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5, y_train_pred)

In [ ]:
recall_score(y_train_5, y_train_pred)

-   Harmonic Mean (of precision and recall) or F1 score
-   F1 = 2/(1/precision  +  1/recall) = 2(precision*recall)/(precision+recall) = TP / ( TP  +   (FN + FP)/2)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

*Precision/Recall Tradeoff*

In [ ]:
y_scores = sgd_clf.decision_function([some_digit])
y_scores

In [ ]:
threshold = 0 #SGDClassifier default
y_some_digit_pred = (y_scores>threshold)

In [ ]:
threshold = 8000
y_some_digit_pred = (y_scores>threshold)
y_some_digit_pred

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

from sklearn.metrics import precision_recall_curve
precisions, recalls, threshold = precision_recall_curve(y_train_5, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, threshold):
    plt.plot(threshold, precisions[:-1], "b--", label="Precision")
    plt.plot(threshold, recalls[:-1], "g-", label = "Recall")
plot_precision_recall_vs_threshold(precisions, recalls, threshold)
plt.show()


In [ ]:
plt.plot(recalls,precisions)

In [ ]:
threshold_90_precision = threshold[np.argmax(precisions>=.90)]
y_train_pred_90 = (y_scores >= threshold_90_precision)
precision_score(y_train_5, y_train_pred_90)

In [ ]:
recall_score(y_train_5, y_train_pred_90)

*ROC Curve*

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, threshold = roc_curve(y_train_5, y_scores)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label = label)
    plt.plot([0,1], [0,1], 'k--')

plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_scores)

-   Use Precision/Recall curve when the positive class is rare or when you care more abt the false positives than false negatives, and ROC curve otherwise.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, method='predict_proba')

y_scores_forest = y_probas_forest[:, 1] #score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

plt.plot(fpr, tpr, "b:", label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.show()

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)

-   Find precision and recall scores: 

**Multiclass Classification**

In [ ]:
sgd_clf.fit(X_train, y_train)
sgd_clf.predict([some_digit])

In [ ]:
some_digit_scores = sgd_clf.decision_function([some_digit])
some_digit_scores

In [ ]:
np.argmax(some_digit_scores)

In [ ]:
sgd_clf.classes_

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(X_train, y_train)
ovo_clf.predict([some_digit])


In [ ]:
len(ovo_clf.estimators_)

In [ ]:
forest_clf.fit(X_train, y_train)
forest_clf.predict([some_digit])


In [ ]:
forest_clf.predict_proba([some_digit])

In [ ]:
cross_val_score(sgd_clf,X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
#Always use unless I'm experience enough to know when not to
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv = 3, scoring="accuracy")

**Error Analysis**

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap = plt.cm.gray)
plt.show()

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx/row_sums

In [ ]:
np.fill_diagonal(norm_conf_mx,0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()

**Multilable Classification**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train>=7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

In [ ]:
knn_clf.predict([some_digit])

In [ ]:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score(y_multilabel, y_train_knn_pred, average="macro")

**Multioutput Classification**

In [ ]:
noise = np.random.randint(0,100, (len(X_train), 784))
X_train_mod = X_train+noise
noise = np.random.randint(0,100, (len(X_train), 784))
X_test_mod = X_test+noise
y_train_mod = X_train
y_test_mod = X_test
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)

In [48]:
from sklearn.model_selection import GridSearchCV
grid_params = {
    'n_neighbors': [3,4,5,11,19],
    'weights': ['uniform', 'distance']
}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose=1,cv=3, n_jobs=4)

gs_result = gs.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


In [ ]:
gs.best_params_

In [ ]:
gs.best_score_


In [ ]:
from sklearn.metrics import accuracy_score
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)